This is the ED code of $N_1 \times N_2$ Kitaev model. Here we use scipy.sparse to generate the Hamiltonian matrix.

\section{Kitaev Hamiltonian}

\begin{equation}
    H_K = - \sum_{<j,k>_\alpha} J_\alpha S_j^\alpha S_k^\alpha
\end{equation}
where $\alpha = x, y, z$.

In [106]:
import numpy as np
import scipy.sparse as sps
from scipy.sparse import identity
from scipy.sparse.linalg import eigsh

def index(N1, N2, c, r, atom = 'A'):
    '''Calculate the index of the atoms in the (r,c) unit cell'''
    c = c % N1
    r = r % N2
    
    n = r * N1 + c
    if atom == 'A':
        ind = 2 * n
    elif atom == 'B':
        ind = 2 * n + 1
    else:
        return print('ERROR: The atom index should be A or B.')
    return ind

def KitaevRhom_sparse(J, N1, N2): #default periodic boundary condition
    
    SiteNum = N1 * N2 * 2
    dim = pow(2,SiteNum)
    Hk = sps.csc_matrix((dim,dim))
    
    # define Pauli matrices
    sx = sps.csc_matrix(0.5 * np.array([[0,1],[1,0]]))
    sy = sps.csc_matrix(0.5 * np.array([[0,-1j],[1j,0]]))
    sz = sps.csc_matrix(0.5 * np.array([[1,0],[0,-1]]))
    
    # interactions
    SxSx = J[0] * sps.kron(sx,sx)
    
    SySy = J[1] * sps.kron(sy,sy)
    
    midy_b = index(N1, N2, -1, 0, atom = 'B') - index(N1, N2, 0, 0, atom = 'A')
    SySy_b = J[1] * sps.kron(sy, sps.kron(identity(pow(2, midy_b - 1)), sy))
                      
    midz = index(N1, N2, 0, 1, atom = 'A') - index(N1, N2, 0, 0, atom = 'B')
    SzSz = J[2] * sps.kron(sz, sps.kron(identity(pow(2, midz - 1)), sz))
    
    midz_b = index(N1, N2, 0, -1, atom = 'B') - index(N1, N2, 0, 0, atom = 'A')
    SzSz_b = J[2] * sps.kron(sz, sps.kron(identity(pow(2, midz_b - 1)), sz))
        
    # define Kitaev Hamiltonian
    for r in range(N2):
        for c in range(N1):
            n = index(N1, N2, c, r, atom = 'A')
            '''Sx - Sx'''
            Hk += sps.kron(identity(pow(2,n)), sps.kron(SxSx, identity(pow(2,SiteNum - n - 2))))
            
            '''Sy - Sy'''
            if c == 0:
                Hk += sps.kron(identity(pow(2,n)), sps.kron(SySy_b, identity(pow(2, SiteNum - n - midy_b -1 ))))
            else:
                Hk += sps.kron(identity(pow(2,n-1)), sps.kron(SySy, identity(pow(2, SiteNum - n - 1))))
    
            '''Sz-Sz'''
            if r == 0:
                Hk += sps.kron(identity(pow(2,n)), sps.kron(SzSz_b, identity(pow(2, SiteNum - n - midz_b - 1))))
            else:
                Hk += sps.kron(identity(pow(2,n - midz)), sps.kron(SzSz, identity(pow(2, SiteNum - n - 1))))
    return Hk


In [107]:
J = np.ones(3)
N1 = 2
N2 = 3
SiteNum = N1 * N2 * 2
Hk = KitaevRhom_sparse(J, N1, N2)
from scipy.sparse.linalg import eigsh
e,v = eigsh(Hk, k = 5,which = 'SA')
print(e/SiteNum)


[-0.20417256 -0.20417256 -0.20417256 -0.19523157 -0.20417256]


In [75]:
A = sps.csr_matrix(np.array([[0, 2], [5, 0]]))
B = sps.csr_matrix(np.array([[1, 2], [3, 4]]))
del C
C = sps.csc_matrix((4,4))
C += sps.kron(A,B)
print(C)

  (2, 0)	5.0
  (3, 0)	15.0
  (2, 1)	10.0
  (3, 1)	20.0
  (0, 2)	2.0
  (1, 2)	6.0
  (0, 3)	4.0
  (1, 3)	8.0
